In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver

# ***********************************PLEASE READ***********************************
#### This will open an extra chrome tab just used to parse some of the data that I couldnt get normally from the website directly because it parsed back "None" Values for some elements even though it existed, therefor I couldn't get all the data I wanted to collect for this project. the chrome tab allows me read those without restrictions and it solves the "None" Value problem

## IMPORTANT: 
### 1) Please dont close the tab or the project will break, you can close the tab when you stop running the code.
### 2) Please when you run the app, make sure to STOP RUNNING the code within a few seconds, as the code will continue to run until its done with all the pages. It has to loop through 33,545 titles which can take a long time.

In [56]:
#test code
"""
*****************************************************PLEASE READ*****************************************************
This will open an extra chrome tab just used to parse some of the data that I couldnt get normally from the website 
directly because it parsed back "None" Values for some elements even though it existed, therefor I couldn't get all the data 
I wanted to collect for this project. the chrome tab allows me read those without restrictions and it solves 
the "None" Value problem

IMPORTANT: Please dont close the tab or the project will break, the tab will close automatically when it is done loading.
"""
driver = webdriver.Chrome()
url = 'https://www.imdb.com/search/title/?title_type=video_game&sort=user_rating,desc'
# driver.maximize_window()
driver.get(url)
content = driver.page_source.encode('utf-8').strip()
""""""
soup = BeautifulSoup(content,"html.parser")
#creates pandas dataframe 
def data_create():
    data = {
        'title': [],
        'release_date': [],
        'maturity_rating': [],
        'genre': [],
        'rating': [],
        'directors': [],
        'stars': [],
        'votes': []
    }
    global df
    df = pd.DataFrame(data= data)
data_create()


#func used to extract data by inputting the soup, the thing you want to find => "selector", and class => "class_selector"
def extract_data(soup, selector, class_selector):
    try:
        return soup.find(selector, class_= class_selector).text.strip()
    except AttributeError:
        return None

#continues looping till its done with finding a "next" link
try: 
    while True:
        #a big section that contains all the data of the videogame title
        game_info = soup.find_all("div", class_="lister-item-content")

        for i in game_info:
            #gets game name
            try:
                game_name = (i.find("h3", class_="lister-item-header")).a.text.strip()
            except AttributeError:
                game_name = None

            #gets release date
            try:
                game_release_date = extract_data(i, "span", "lister-item-year text-muted unbold")
                game_release_date = game_release_date.replace("(", "")
                game_release_date = game_release_date.replace(")", "")
                game_release_date = game_release_date.replace("I", "")
                game_release_date = int(game_release_date.replace("Video Game", "").strip())
            except:
                game_release_date = 0
            
            #gets maturity rating
            game_maturity = extract_data(i, "span", "certificate")

            #gets genre and split it into an array
            try:
                game_genre = extract_data(i, "span", "genre")
                if game_genre.find(",")!=-1:
                    game_genre = np.array(list(game_genre.split(",")))
                else:
                    game_genre = np.array(game_genre)
            except AttributeError:
                game_genre = None

            #get rating
            try:
                game_rating = float(i.find("div", class_="inline-block ratings-imdb-rating").strong.text.strip())
            except AttributeError:
                game_rating = 0

            #get Director & Stars and will split them, and will put "None" in values that dont exist
            #get both
            game_director_and_stars = extract_data(i, "p", "")
            game_director = None
            game_stars = None
            #if both director and star data exist for the title
            if game_director_and_stars.find("|")!=-1:
                game_director_full_text, separator, game_stars_full_text = game_director_and_stars.partition("|")

                game_director_full_text = game_director_full_text.strip()
                game_director_full_text = ''.join(game_director_full_text.splitlines())
                game_stars_full_text = game_stars_full_text.strip()
                game_stars_full_text = ''.join(game_stars_full_text.splitlines())

                director_text, separator, game_director = game_director_full_text.partition(":")
                star_text, separator, game_stars = game_stars_full_text.partition(":")

                #finds if there are many Directors/Stars or just one
                #there are multiple Directors
                if game_director.find(",")!=1:
                    game_director = np.array(list(game_director.split(",")))
                #there is only 1 Director
                else:
                    game_director = np.array(list(game_director))
                #there are multiple Stars
                if game_stars.find(",")!=1:
                    game_stars = np.array(list(game_stars.split(",")))
                #there is only 1 Star
                else:
                    game_stars = np.array(list(game_stars))

            #didnt both Director and Star together, will try to find if only one of them exists        
            #if found Director
            elif game_director_and_stars.find("Director")!=-1:
                game_director_and_stars = ''.join(game_director_and_stars.splitlines())
                director_text, separator, game_director = game_director_and_stars.partition(":")
                game_director = game_director.replace("\n", "")
                #finds if there are many Directors or just one
                #there are multiple Directors
                if game_director.find(",")!=1:
                    game_director = np.array(list(game_director.split(",")))
                #there is only 1 Director
                else:
                    game_director = np.array(list(game_director))
                game_stars = None

            #if found Star
            elif game_director_and_stars.find("Star")!=-1:
                star_text, separator, game_stars = game_director_and_stars.partition(":")
                game_stars = game_stars.replace("\n", "")
                #finds if there are many Stars or just one
                #there are multiple Stars
                if game_stars.find(",")!=1:
                    game_stars = np.array(list(game_stars.split(",")))
                #there is only 1 Star
                else:
                    game_stars = np.array(list(game_stars))
                game_director = None
            else:
                game_director = None
                game_stars = None


            #get votes
            try: 
                game_votes = extract_data(i, "p", "sort-num_votes-visible")
                game_votes = ''.join(game_votes.splitlines())
                deleted_text, separator, game_votes = game_votes.partition(":")
                game_votes = game_votes.replace(",", "").strip()
                game_votes = int(game_votes)
            except:
                game_votes = 0

            #adds game details to dataframe
            df.loc[len(df.index)] = [game_name, game_release_date, game_maturity, game_genre, game_rating, game_director, game_stars, game_votes]
    #         display(df)


        #find button to click on next page 
        uncompleted_next_page_url = soup.find("a", class_="lister-page-next next-page").get("href")
        completed_next_page_url = "https://www.imdb.com/" + uncompleted_next_page_url

        print(completed_next_page_url)

        #replaces old url to completed_next_page_url
        url = completed_next_page_url
        driver.get(url)
        content = driver.page_source.encode('utf-8').strip()

        soup = BeautifulSoup(content, "html.parser")

    driver.quit()
except Exception as e:
    print(e)
    print("Scrapping Complete.")

D:\Anaconda\lib\site-packages\pandas\core\dtypes\cast.py:948: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=51&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=101&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=151&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=201&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=251&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=301&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=351&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=401&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=451&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating

https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=4001&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=4051&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=4101&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=4151&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=4201&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=4251&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=4301&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=4351&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=4401&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=u

https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=7951&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=8001&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=8051&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=8101&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=8151&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=8201&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=8251&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=8301&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=8351&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=u

https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WzYuMCw2NzIyOCwidHQwMTI0MTQyIiwxMTM1MV0%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WzYuMCwzNjU3NTMsInR0Njk5MjkyMCIsMTE0MDFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WzYuMCw4NzI2MzgsInR0MjMxNzc4NiIsMTE0NTFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WzYuMCwyMzQxODg2LCJ0dDAyODQ1MDgiLDExNTAxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WzYuMCwyMzc3OTM0LCJ0dDEyMjM0NDkyIiwxMTU1MV0%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WzYuMCwyNDUxOTk0LCJ0dDE0MDY5MjcyIiwxMTYwMV0%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WzUuOSwxNjczNzcsInR0NzQ0NzEyNiIsMTE2NTFd&ref_=adv_nx

...

https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0MDE3MjksInR0MjIwNTYxNyIsMjIyNTFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0MDIxMzksInR0MTIxOTc2NiIsMjIzMDFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0MDI1OTcsInR0NjcyODIzNiIsMjIzNTFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0MDMwNTAsInR0MTgwODA5MCIsMjI0MDFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0MDM1MjksInR0NTQ2OTU5OCIsMjI0NTFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0MDQwMjAsInR0MTk1OTM5OSIsMjI1MDFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=Wy

https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0MjU1MTYsInR0MTYwNjMyOTIiLDI0OTAxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0MjYwMDEsInR0MDE3NzM5NSIsMjQ5NTFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0MjY1MjcsInR0MTg5MDM2OCIsMjUwMDFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0MjY5NjYsInR0MTUxMjE4ODgiLDI1MDUxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0Mjc0MzUsInR0MTUxNTYxODIiLDI1MTAxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0Mjc5MDksInR0NTM3MTg2MCIsMjUxNTFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=u

https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0NDk3MzMsInR0MTAxMTE0MTQiLDI3NTUxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0NTAzMDUsInR0OTI2MTI5OCIsMjc2MDFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0NTA3ODcsInR0MjI4NzAzODAiLDI3NjUxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0NTEyNDYsInR0MjA4NjQ3MjYiLDI3NzAxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0NTE1ODYsInR0MTY5NzA2NTQiLDI3NzUxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0NTIwNzQsInR0MTk3OTc4MTQiLDI3ODAxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=v

https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0NzM5OTEsInR0MDkyNTI5MSIsMzAyMDFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0NzQ1MTgsInR0MjIyNDg2MCIsMzAyNTFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0NzQ5MjcsInR0MjM1MTgzMCIsMzAzMDFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0NzU0NTYsInR0MTI1MDc1OSIsMzAzNTFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0NzU4NzksInR0NDgzMzY3OCIsMzA0MDFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0NzYzMjcsInR0MTIxMDQyNDIiLDMwNDUxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&

https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0OTgxMzUsInR0MjM4NzM5MTgiLDMyODUxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0OTg1NDAsInR0MjU0MjYyMCIsMzI5MDFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0OTg5MjgsInR0MjIwMDc2MTIiLDMyOTUxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0OTk0ODEsInR0NzQ0NTE5NiIsMzMwMDFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI0OTk4OTQsInR0MTM0NjM1NTAiLDMzMDUxXQ%3D%3D&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&after=WyItSW5maW5pdHkiLDI1MDAyNzAsInR0MDM2NTcxOCIsMzMxMDFd&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=u

In [58]:
df

,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Last of Us: Part I,2022,M,"[Action, Adventure, Drama]",9.8,"[Matthew Gallant, Bruce Straley]","[Ashley Johnson, Troy Baker, Hana Hayes, Je...",3495
1,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,"[Action, Adventure, Drama]",9.8,[Konrad Tomaszkiewicz],"[Doug Cockle, MyAnna Buring, Antonia Bernath...",8789
2,The Last of Us,2013,M,"[Action, Adventure, Drama]",9.7,"[Neil Druckmann, Bruce Straley]","[Troy Baker, Ashley Johnson, Hana Hayes, Je...",68864
3,God of War: Ragnarök,2022,M,"[Action, Adventure, Drama]",9.7,[Eric Williams],"[Christopher Judge, Sunny Suljic, Danielle B...",11539
4,Red Dead Redemption II,2018,M,"[Action, Adventure, Crime]",9.7,None,"[Benjamin Byron Davis, Roger Clark, Rob Wiet...",42796
...,...,...,...,...,...,...,...,...
33514,"Austin Powers: Oh, Behave!",0,T,Action,0.0,None,None,0
33515,Austin Powers: Welcome to My Underground Lair!,0,T,Action,0.0,None,None,0
33516,Chippy,2019,E,"[Action, Adventure, Sci-Fi]",0.0,None,None,0
33517,Blood Bowl 3,2023,None,None,0.0,None,None,0


In [59]:
#Save CSV
df.to_csv('ToCSV.csv', index=False)